In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, os, glob

In [2]:
# if there is no TRAINSET and TESTSET folder, create them
if not os.path.exists('TRAINSET'):
    os.makedirs('TRAINSET')
if not os.path.exists('TESTSET'):
    os.makedirs('TESTSET')
    

In [3]:
def wekly_cvt_df(df):

    df = df.apply(pd.to_numeric, errors='coerce')

    df.index = pd.to_datetime(df.index)

    df = df.loc[~df.index.isin([pd.NaT])]

    df = df.loc[~df.index.duplicated(keep='last')]

    # drop column if the column type is object
    # df = df.loc[:, df.dtypes != 'object']

    # drop columns when there are more than 20% of NaN values
    # df = df.loc[:, df.isnull().sum() < len(df)*0.2]

    # convert index name to 'date' from raw index name.
    df.index.name = 'date'

    return df


In [4]:
# control dataframe

def daily_cvt_df(df):

    #convert type to numeric all columns
    df = df.apply(pd.to_numeric, errors='coerce')
    
    df.index = pd.to_datetime(df.index)

    df = df.loc[~df.index.isin([pd.NaT])]

    df = df.loc[~df.index.duplicated(keep='last')]

    # drop column if the column type is object
    # df = df.loc[:, df.dtypes != 'object']

    # drop columns when there are more than 20% of NaN values
    # df = df.loc[:, df.isnull().sum() < len(df)*0.1]

    # make min, max, mean of each column
    # 1. make column min, max, mean of each column
    # 2. use rolling window to make min, max, mean of each column
    # 3. concat 1 and 2
    # 4. drop columns when there are more than 20% of NaN values
    # 5. window size is 7
    # 6. set the column name as '_min', '_max', '_mean'
    # 7. drop raw columns
    # df = pd.concat([df, df.rolling(7).min().add_suffix('_min'), df.rolling(7).max().add_suffix('_max'), df.rolling(7).mean().add_suffix('_mean')], axis=1)

    # sample date from given index list. If the index is not in the dataframe, drop it.
    # df = df.loc[df.index.isin(index)]

    # df = df.dropna(axis=0)

    # convert index name to 'date' from raw index name.
    df.index.name = 'date'

    return df

In [5]:
# read ./DATA/지점 제원.xlsx
df = pd.read_excel('./DATA/지점 제원.xlsx', index_col=[0,1])

df.head()

조류  주간  일간    기상        수위   댐   보
수계  no                                       
낙동강 1    물금메리  물금  상동   양산시   김해시_월촌리 NaN  함안
    3   강정고령보  성주  다산  대구_기  성주군_성주대교 NaN  고령
    4      해평  강정  해평    구미   구미시_일선교 NaN  구미
    5     상주보  도남  회상    상주   상주시_강창교 NaN  상주
    6     낙단보  낙단  신암    상주   의성군_낙단교 NaN  낙단

In [6]:
# df.head()

folder_atm = './DATA/기상'
folder_alg = './DATA/조류'
folder_wat = './DATA/수위'
folder_wek = './DATA/주간'
folder_day = './DATA/일간'
folder_wee = './DATA/댐보'

In [94]:
# use atm column : 평균기온,최저기온,최저기온시각,최고기온,일강수량,최대순간풍속,최대순간풍속풍향방위,최대순간풍속시각,최대풍속,최대풍속풍향방위,평균풍속,최다풍향방위,평균이슬점온도,최소상대습도,평균상대습도,평균증기압,평균현지기압,평균지면온도,최저초상온도
# day_atm_columns = ['평균기온','최저기온','최고기온','일강수량','최대풍속','최대풍속풍향방위','평균풍속','최다풍향방위','평균이슬점온도','최소상대습도','평균상대습도','평균증기압','평균현지기압','평균지면온도','최저초상온도']
day_atm_columns = ['평균기온','최저기온','최고기온','일강수량','최대풍속','평균풍속']

# use wear column : 보방류량_DAY, 보유입량_DAY,보자체유입량_DAY,보저수량_DAY,보저수위_DAY
day_wee_columns = ['보방류량_DAY','보유입량_DAY','보저수량_DAY','보저수위_DAY']

# use water level columns : 수위_DAY, 유량_DAY
day_wl_columns = ['수위_DAY','유량_DAY']

# water quality day column : 수온, 수소이온농도, 전기전도도, 용존산소, 탁도, 총유기탄소, 총질소, 총인, 클로로필
# day_wq_columns = ['수온','수소이온농도','전기전도도','용존산소','탁도','총유기탄소','클로로필']#, '총질소','총인']
day_wq_columns = ['수온','용존산소','총유기탄소','클로로필']#, '총질소','총인']

# algee data columns : 수온,pH,DO,투명도,탁도,Chl-a,유해남조류세포수
wek_alg_columns = ['수온','pH','DO','투명도','탁도','Chl-a ','유해남조류세포수']

In [154]:
df.to_csv('./model_attn/df.csv', encoding='cp949')

In [146]:

# df.head()

folder_atm = './DATA/기상'
folder_alg = './DATA/조류'
folder_wat = './DATA/수위'
folder_wek = './DATA/주간'
folder_day = './DATA/일간'
folder_wee = './DATA/댐보'


# define start time with timestamp format
start_date = pd.Timestamp('2012-01-01')
end_date   = pd.Timestamp('2023-01-01')

all_df = pd.DataFrame()

all_df_list = []
print(df.columns)
for i, idx in enumerate(df.index):
    

    print(idx, df.loc[idx]['조류'], df.loc[idx]['기상'], df.loc[idx]['수위'], df.loc[idx]['주간'], df.loc[idx]['일간'])
    
    
    alg_df = pd.read_csv(folder_alg + '/' + df.loc[idx]['조류'] + '.csv', index_col=0, encoding='cp949', parse_dates=True)
    atm_df = pd.read_csv(folder_atm + '/' + df.loc[idx]['기상'] + '.csv', index_col=0, encoding='cp949', parse_dates=True)
    wat_df = pd.read_csv(folder_wat + '/' + df.loc[idx]['수위'] + '.csv', index_col=0, encoding='cp949', parse_dates=True)
    # wek_df = pd.read_csv(folder_wek + '/' + df.loc[idx]['주간'] + '.csv', index_col=0, encoding='cp949')
    day_df = pd.read_csv(folder_day + '/' + '%s_'%idx[0] + df.loc[idx]['일간'] + '.csv', index_col=0, encoding='cp949', parse_dates=True)
    wee_df = pd.read_csv(folder_wee + '/' +  df.loc[idx]['보'] + '.csv', index_col=0, encoding='cp949', parse_dates=True)

    alg_df = alg_df[wek_alg_columns]

    #append _WEK to column name and maintain column name only '유해남조류세포수' column. And interpolate nan value with linear method
    alg_df.columns = ['%s_WEK'%col if col != '유해남조류세포수' else col for col in alg_df.columns]

    # reindex with date range
    alg_df = alg_df[~alg_df.index.duplicated(keep='last')]
    alg_df = alg_df.reindex(pd.date_range(start_date, end_date, freq='D'))

    # interpolate nan value with linear method. if 'WEK' in col
    # condition 1 : the period of nan value is less than 14 rows : interpolate nan value with linear method
    # condtionn 2 : the period of nan value is more than 14 rows : no action
    for col in alg_df.columns:
        if 'WEK' in col:
            alg_df[col] = alg_df[col].interpolate(method='linear', limit=14, limit_direction='both')
    
    
    # drop columns which has nan value more than 20% and except '유해남조류세포수' column of alg_df
    # for col in alg_df.columns:
    #     if col != '유해남조류세포수' and alg_df[col].isna().sum() > 0.2*len(alg_df):
    #         alg_df.drop(col, axis=1, inplace=True)

    atm_df = atm_df[day_atm_columns]
    # fill nan value with 0 only '일강수량' column
    atm_df['일강수량'] = atm_df['일강수량'].fillna(0)

    wat_df = wat_df[day_wl_columns]
    day_df = day_df[day_wq_columns]
    wee_df = wee_df[day_wee_columns]

    print(alg_df['유해남조류세포수'].dropna(axis=0).shape)

    # # convert to datetime format and set index of above dataframes
    # alg_df.index = pd.to_datetime(alg_df.index)
    # atm_df.index = pd.to_datetime(atm_df.index)
    # wat_df.index = pd.to_datetime(wat_df.index)
    # wek_df.index = pd.to_datetime(wek_df.index)
    # day_df.index = pd.to_datetime(day_df.index)
    # wee_df.index = pd.to_datetime(wee_df.index)

    # alg_df = alg_df.loc[~alg_df.index.isin([pd.NaT])]

    day_df_list = [daily_cvt_df(edf) for edf in [atm_df, wat_df, day_df, wee_df]]

    wek_df_list = [wekly_cvt_df(edf) for edf in [alg_df]]

    # concat tow list day_df_list and wek_df_list
    df_list = day_df_list + wek_df_list
    
    # # concat dataframes
    condf = pd.concat(df_list, axis=1)

    # slice index <= 2021-12-31
    # condf = condf.loc[condf.index <= '2021-12-31']

    # drop columns when there are more than 20% of NaN values
    # condf = condf.loc[:, condf.isnull().sum() < len(condf)*0.2]

    # if condf.shape[0] > 300:
    # make row with nan values from start_date to end_date
    condf = condf.reindex(pd.date_range(start_date, end_date, freq='D'))

    # last line of condf is need to convert to nan values
    condf.iloc[-1] = np.nan

    # drop duplicated rows
    # condf = condf[~condf.index.duplicated(keep='last')]
    # condf = condf.dropna(axis=0)

    # save to csv file in ./POST_DATA/ folder
    condf.to_csv('./POST_DATA/%s_%s.csv'%(idx[0], df.loc[idx]['조류']), encoding='cp949')


    # join all_df and condf using join method
    # all_df = all_df.join(condf, how='outer')


    all_df_list.append(condf)


Index(['조류', '주간', '일간', '기상', '수위', '댐', '보'], dtype='object')
('낙동강', 1) 물금메리 양산시 김해시_월촌리 물금 상동
(145,)
('낙동강', 3) 강정고령보 대구_기 성주군_성주대교 성주 다산
(627,)
('낙동강', 4) 해평 구미 구미시_일선교 강정 해평
(334,)
('낙동강', 5) 상주보 상주 상주시_강창교 도남 회상
(543,)
('낙동강', 6) 낙단보 상주 의성군_낙단교 낙단 신암
(557,)
('낙동강', 7) 구미보 구미 구미시_일선교 산곡 도개
(571,)
('낙동강', 8) 칠곡보 구미 칠곡군_호국의다리 칠곡 칠곡
(604,)
('낙동강', 9) 강정고령 대구_기 대구시_매곡리 다사 다산
(355,)
('낙동강', 10) 달성보 대구_기 대구시_성하리 논공 고령
(679,)
('낙동강', 11) 합천창녕보 합천 합천군_율지교 덕곡 적포
(688,)
('낙동강', 12) 창녕함안보 의령군 함안군_계내리 함안 청암
(664,)


In [147]:
# concat all_df_list
all_df = pd.concat(all_df_list, axis=0)

# reset index
all_df = all_df.reset_index()

# save all_df to csv file
all_df.to_csv('./POST_DATA/all_data.csv', encoding='cp949', index=False)

In [148]:
# compare raw shape and drop nan shape of all_df
print(all_df.shape, all_df.dropna(axis=0).shape)

(44209, 24) (543, 24)


In [151]:
# calculte % of nan values in each columns of condf
nan_df = pd.DataFrame(all_df.isnull().sum()/len(all_df)*100, columns=['% of NaN'])

In [152]:
nan_df

,% of NaN
index,0.000000
평균기온,29.745075
최저기온,29.704359
최고기온,29.702097
일강수량,29.688525
최대풍속,29.738289
평균풍속,29.751860
수위_DAY,0.599425
유량_DAY,3.772987
수온,24.712163
